<a href="https://colab.research.google.com/github/ShrikantKGIT/general/blob/main/Python_Assembly_like_Interpreter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This Python script implements a simple, register-based interpreter that simulates how a CPU executes assembly language. It parses a custom instruction set for data manipulation (mov, add), comparison (cmp), and control flow (jne), using an instruction pointer and labels to manage program execution. This project demonstrates a foundational understanding of computer architecture and low-level code processing.**

In [1]:
import re

class AssemblyInterpreter:
    """
    An interpreter for a simple, custom assembly-like language.

    This interpreter features:
    - A set of registers for data storage.
    - An instruction pointer to control execution flow.
    - Support for labels and conditional/unconditional jumps.
    - A simple instruction set (MOV, ADD, SUB, CMP, JMP, JE, JNE, PRN).
    """

    def __init__(self):
        """Initializes the interpreter's state."""
        self.registers = {'eax': 0, 'ebx': 0, 'ecx': 0, 'edx': 0}
        self.ip = 0  # Instruction Pointer
        self.zero_flag = False  # A simple flag for comparison results
        self.instructions = []
        self.labels = {}

    def _parse_operand(self, operand):
        """
        Parses an operand to determine if it's a literal value or a register.

        Returns:
            The integer value of the literal or the value stored in the register.
        """
        operand = operand.strip()
        if operand in self.registers:
            return self.registers[operand]
        try:
            return int(operand)
        except ValueError:
            raise ValueError(f"Invalid operand: {operand}")

    def load_program(self, code):
        """
        Parses the assembly code, identifying labels and instructions.

        Args:
            code (str): A string containing the assembly-like program.
        """
        lines = [line.strip() for line in code.strip().split('\n')]

        # First pass: identify labels and clean up instructions
        temp_instructions = []
        for line in lines:
            # Ignore comments (starting with ';') and empty lines
            if not line or line.startswith(';'):
                continue

            # Check for labels (e.g., "loop:")
            match = re.match(r'^([a-zA-Z_][a-zA-Z0-9_]*):$', line)
            if match:
                label_name = match.group(1)
                # A label points to the *next* instruction
                self.labels[label_name] = len(temp_instructions)
            else:
                temp_instructions.append(line)

        self.instructions = temp_instructions

    def run(self):
        """Executes the loaded program until completion."""
        print("--- Running Assembly Program ---")
        while self.ip < len(self.instructions):
            current_ip = self.ip
            instruction_line = self.instructions[self.ip]
            parts = re.split(r'[ ,]+', instruction_line, maxsplit=2)
            opcode = parts[0].lower()
            operands = parts[1:] if len(parts) > 1 else []

            print(f"Executing [IP={current_ip}]: {instruction_line}")

            # Assume we will go to the next instruction unless a jump occurs
            next_ip = self.ip + 1

            # --- Instruction Set Implementation ---
            try:
                if opcode == 'mov': # mov eax, 10 or mov eax, ebx
                    reg_dest = operands[0]
                    val_src = self._parse_operand(operands[1])
                    self.registers[reg_dest] = val_src

                elif opcode == 'add': # add eax, 5
                    reg_dest = operands[0]
                    val_src = self._parse_operand(operands[1])
                    self.registers[reg_dest] += val_src

                elif opcode == 'sub': # sub eax, 5
                    reg_dest = operands[0]
                    val_src = self._parse_operand(operands[1])
                    self.registers[reg_dest] -= val_src

                elif opcode == 'cmp': # cmp eax, 10
                    val1 = self._parse_operand(operands[0])
                    val2 = self._parse_operand(operands[1])
                    self.zero_flag = (val1 == val2)
                    print(f"  Comparison: {val1} == {val2} -> ZeroFlag={self.zero_flag}")

                elif opcode == 'jmp': # jmp my_label
                    label = operands[0]
                    next_ip = self.labels[label]

                elif opcode == 'je': # je my_label (Jump if Equal)
                    if self.zero_flag:
                        label = operands[0]
                        next_ip = self.labels[label]

                elif opcode == 'jne': # jne my_label (Jump if Not Equal)
                    if not self.zero_flag:
                        label = operands[0]
                        next_ip = self.labels[label]

                elif opcode == 'prn': # prn eax (Custom print instruction)
                    val = self._parse_operand(operands[0])
                    print(f"Output: {val}")

                else:
                    print(f"Error: Unknown opcode '{opcode}'")
                    return

            except (ValueError, KeyError, IndexError) as e:
                print(f"Error on line {current_ip}: {instruction_line}\n  -> {e}")
                return

            self.ip = next_ip

        print("--- Execution Finished ---")


# --- Example Usage ---

if __name__ == "__main__":
    # This program initializes a counter in 'eax' to 5,
    # then loops, printing and decrementing the counter
    # until it reaches 0.

    assembly_code = """
    ; Simple countdown loop
    mov eax, 5      ; Set counter to 5

    loop_start:
        prn eax     ; Print the current value of eax
        sub eax, 1  ; Decrement the counter
        cmp eax, 0  ; Compare the counter with 0
        jne loop_start ; Jump back to loop_start if not equal to 0

    mov ebx, 99     ; This line is reached after the loop finishes
    prn ebx
    """

    # Create an interpreter instance
    interpreter = AssemblyInterpreter()

    # Load and run the program
    interpreter.load_program(assembly_code)
    interpreter.run()

    # Inspect the final state of the registers
    print("\nFinal Registers:", interpreter.registers)


--- Running Assembly Program ---
Executing [IP=0]: mov eax, 5      ; Set counter to 5
Error on line 0: mov eax, 5      ; Set counter to 5
  -> Invalid operand: 5      ; Set counter to 5

Final Registers: {'eax': 0, 'ebx': 0, 'ecx': 0, 'edx': 0}
